In [ ]:
%pip install \
    transformers \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    bitsandbytes  \
    scikit-learn \
    peft --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.2 MB/s eta 0:00:00
   ━

In [ ]:


import torch
import pandas as pd
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding)

import bitsandbytes as bnb

import evaluate
import numpy as np

import random



In [ ]:
#copy toj=ken from https://huggingface.co/settings/tokens
import huggingface_hub
hf_token = 'hf_...' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

In [ ]:
dataset = load_dataset("MVesalA/Persian-NER-GUI-Classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/526 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/383k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/388k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12788 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2740 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2741 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'existence'],
        num_rows: 12788
    })
    validation: Dataset({
        features: ['text', 'existence'],
        num_rows: 2740
    })
    test: Dataset({
        features: ['text', 'existence'],
        num_rows: 2741
    })
})

In [ ]:
model_id  = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(f' Vocab size of the model {model_id}: {len(tokenizer.get_vocab())}')

 Vocab size of the model HooshvareLab/bert-fa-base-uncased: 100000


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,  max_length=512)

In [ ]:
tokenized_NER = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/12788 [00:00<?, ? examples/s]

Map:   0%|          | 0/2740 [00:00<?, ? examples/s]

Map:   0%|          | 0/2741 [00:00<?, ? examples/s]

In [ ]:
tokenized_NER

DatasetDict({
    train: Dataset({
        features: ['text', 'existence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12788
    })
    validation: Dataset({
        features: ['text', 'existence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2740
    })
    test: Dataset({
        features: ['text', 'existence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2741
    })
})

In [ ]:
def preprocess_function(example):
    # حذف token_type_ids و تغییر نام existence به label
    return {
        'text': example['text'],
        'label': example['existence'],
        'input_ids': example['input_ids'],
        'attention_mask': example['attention_mask']
    }

# اعمال تغییرات روی تمام splits
tokenized_NER = tokenized_NER.map(preprocess_function, remove_columns=['token_type_ids', 'existence'])

Map:   0%|          | 0/12788 [00:00<?, ? examples/s]

Map:   0%|          | 0/2740 [00:00<?, ? examples/s]

Map:   0%|          | 0/2741 [00:00<?, ? examples/s]

In [ ]:
tokenized_NER

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 12788
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2740
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2741
    })
})

In [ ]:
for i in range(10):
    print(len(tokenized_NER['train'][i]['input_ids']))

77
31
25
35
46
34
44
32
14
12


In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Convert probabilities to predicted labels
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_use_double_quant=True,  # Use double quantization for potentially higher accuracy (optional)
    bnb_4bit_quant_type="nf4",  # Quantization type (specifics depend on hardware and library)
    bnb_4bit_compute_dtype=torch.bfloat16  # Compute dtype for improved efficiency (optional)
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,  # "HooshvareLab/bert-fa-base-uncased"
    num_labels=2,  # Number of output labels (2 for binary sentiment classification)
    id2label=id2label,  # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id=label2id,  # {"NEGATIVE": 0, "POSITIVE": 1}
    quantization_config=bnb_config  # configuration for quantization
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
def find_linear_names(model):
    """
    This function identifies all linear layer names within a model that use 4-bit quantization.
    Args:
        model (torch.nn.Module): The PyTorch model to inspect.
    Returns:
        list: A list containing the names of all identified linear layers with 4-bit quantization.
    """
    cls = bnb.nn.Linear4bit

    # Set to store identified layer names
    lora_module_names = set()

    # Iterate through named modules in the model
    for name, module in model.named_modules():
        # Check if the current module is an instance of the 4-bit linear layer class
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

        # Special case: remove 'lm_head' if present
        if 'lm_head' in lora_module_names:
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

# Example usage:
modules = find_linear_names(model)
print(modules)

['key', 'dense', 'value', 'query']


In [ ]:
lora_config = LoraConfig(
    r=64,  # Reduction factor (lower r means more parameters in the adapter)
    lora_alpha=32,  # Dimensionality of the adapter projection
    target_modules=modules,  # List of modules to apply the LoRA adapter
    lora_dropout=0.05,  # Dropout rate for the adapter
    bias="none",  # Bias configuration for the adapter
    task_type="SEQ_CLS"  # Task type (sequence classification in this case)
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 10,716,674 || all params: 173,559,556 || trainable%: 6.1746


In [ ]:


training_args = TrainingArguments(
    output_dir="epoch_weights",  # Output directory for checkpoints
    learning_rate=2e-5,  # Learning rate for the optimizer
    per_device_train_batch_size=1,  # Batch size per device
    per_device_eval_batch_size=1,  # Batch size per device for evaluation
    num_train_epochs=5,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    eval_strategy='epoch',  # Evaluate after each epoch
    save_strategy="epoch",  # Save model checkpoints after each epoch
    load_best_model_at_end=True,  # Load the best model based on the chosen metric
    push_to_hub=False,  # Disable pushing the model to the Hugging Face Hub
    report_to="none",  # Disable logging to Weight&Bias
    metric_for_best_model='eval_loss'  # Metric for selecting the best model
)

In [ ]:
early_stop = EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=.0)

In [ ]:
trainer = Trainer(
    model=model,  # The LoRA-adapted model
    args=training_args,  # Training arguments
    train_dataset=tokenized_NER["train"],  # Training dataset
    eval_dataset=tokenized_NER["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer for processing text
    data_collator=data_collator,  # Data collator for preparing batches
    compute_metrics=compute_metrics,  # Function to calculate evaluation metrics
    callbacks=[early_stop]  # Optional early stopping callback
)

trainer.train()

/tmp/ipython-input-2607227898.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.153400,0.115088,0.981029,0.965007,0.961126,0.968919


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.153400,0.115088,0.981029,0.965007,0.961126,0.968919
2,0.074000,0.111988,0.982853,0.968393,0.963855,0.972973
3,0.037000,0.121273,0.983218,0.969251,0.958995,0.979730


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=38364, training_loss=0.14039640753028812, metrics={'train_runtime': 8024.41, 'train_samples_per_second': 7.968, 'train_steps_per_second': 7.968, 'total_flos': 742475974640184.0, 'train_loss': 0.14039640753028812, 'epoch': 3.0})

In [ ]:
peft_model_path="./peft-HooshvareLab-NER"

trainer.model.save_pretrained(peft_model_path)

In [ ]:
tokenizer.save_pretrained(peft_model_path)

('./peft-HooshvareLab-NER/tokenizer_config.json',
 './peft-HooshvareLab-NER/special_tokens_map.json',
 './peft-HooshvareLab-NER/vocab.txt',
 './peft-HooshvareLab-NER/added_tokens.json',
 './peft-HooshvareLab-NER/tokenizer.json')

In [ ]:
def predict(input_text):
    """
    Predicts the sentiment label for a given text input.

    Args:
        input_text (str): The text to predict the sentiment for.

    Returns:
        float: The predicted probability of the text being positive sentiment.
    """
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # Convert to PyTorch tensors and move to GPU (if available)
    with torch.no_grad():
        outputs = model(**inputs).logits  # Get the model's output logits
    y_prob = torch.sigmoid(outputs).tolist()[0]  # Apply sigmoid activation and convert to list
    return np.round(y_prob, 5)  # Round the predicted probability to 5 decimal places

In [ ]:
df_test = pd.DataFrame(dataset['validation'])

df_test['prediction'] = df_test['text'].map(predict)
df_test['y_pred'] = df_test['prediction'].apply(lambda x: np.argmax(x, axis=0))
accuracy = (df_test['y_pred'] == df_test['existence']).mean()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
df_test.head()

Model Accuracy on Test Data: 0.3777


,text,existence,prediction,y_pred
0,نخستین مقاومت اسلامی لبنان در حوزه ادبیات کودک...,0,"[0.59119, 0.55803]",0
1,اما او نتوانست موافقت والدینش را برای خروج از ...,0,"[0.5335, 0.53464]",1
2,بر اساس دستور کار هفتگی مجلس ، قرار بود این اع...,0,"[0.46984, 0.53838]",1
3,در ادامه همین اقدامات مالی مشکوک ، مه آفرید ام...,1,"[0.51741, 0.57726]",1
4,چارلتون حریف اورتون نشد .,0,"[0.45442, 0.60173]",1


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
   peft_model_path, num_labels=2,
    id2label=id2label,
    label2id=label2id,
    quantization_config=bnb_config,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def predict(input_text):
    """
    Predicts the sentiment label for a given text input.

    Args:
        input_text (str): The text to predict the sentiment for.

    Returns:
        float: The predicted probability of the text being positive sentiment.
    """
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # Convert to PyTorch tensors and move to GPU (if available)
    with torch.no_grad():
        outputs = model(**inputs).logits  # Get the model's output logits
    y_prob = torch.sigmoid(outputs).tolist()[0]  # Apply sigmoid activation and convert to list
    return np.round(y_prob, 5)  # Round the predicted probability to 5 decimal places

In [ ]:
df_test = pd.DataFrame(dataset['validation'])

df_test['prediction'] = df_test['text'].map(predict)
df_test['y_pred'] = df_test['prediction'].apply(lambda x: np.argmax(x, axis=0))
accuracy = (df_test['y_pred'] == df_test['existence']).mean()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
df_test.head()

Model Accuracy on Test Data: 0.9825


,text,existence,prediction,y_pred
0,نخستین مقاومت اسلامی لبنان در حوزه ادبیات کودک...,0,"[0.9917, 0.00941]",0
1,اما او نتوانست موافقت والدینش را برای خروج از ...,0,"[0.98926, 0.01073]",0
2,بر اساس دستور کار هفتگی مجلس ، قرار بود این اع...,0,"[0.98975, 0.00916]",0
3,در ادامه همین اقدامات مالی مشکوک ، مه آفرید ام...,1,"[0.00749, 0.98877]",1
4,چارلتون حریف اورتون نشد .,0,"[0.02165, 0.97412]",1


In [ ]:
from getpass import getpass


In [ ]:
hf_token = getpass("Hugging Face: ")
huggingface_hub.login(hf_token)

Hugging Face: ··········


In [ ]:
# peft_model_path = "./peft-cnn-summarization"
# trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)


base_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,  # "HooshvareLab/bert-fa-base-uncased"
    num_labels=2,  # Number of output labels (2 for binary sentiment classification)
    id2label=id2label,  # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id=label2id,  # {"NEGATIVE": 0, "POSITIVE": 1}
    quantization_config=bnb_config  # configuration for quantization
)

peft_model = PeftModel.from_pretrained(base_model, peft_model_path)

merged_model = peft_model.merge_and_unload()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
full_model_path = "./peft-NER-existenceCLS-HooshvareLab"
merged_model.save_pretrained(full_model_path)
tokenizer.save_pretrained(full_model_path)

('./peft-NER-existenceCLS-HooshvareLab/tokenizer_config.json',
 './peft-NER-existenceCLS-HooshvareLab/special_tokens_map.json',
 './peft-NER-existenceCLS-HooshvareLab/vocab.txt',
 './peft-NER-existenceCLS-HooshvareLab/added_tokens.json',
 './peft-NER-existenceCLS-HooshvareLab/tokenizer.json')

In [ ]:
merged_model.push_to_hub("MVesalA/peft-NER-existenceCLS-HooshvareLab")
tokenizer.push_to_hub("MVesalA/peft-NER-existenceCLS-HooshvareLab")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...eCLS-HooshvareLab/model.safetensors:   0%|          | 10.9kB /  199MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MVesalA/peft-NER-existenceCLS-HooshvareLab/commit/cedaf2747522d6d377f09d3665f817f8bfd4560d', commit_message='Upload tokenizer', commit_description='', oid='cedaf2747522d6d377f09d3665f817f8bfd4560d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MVesalA/peft-NER-existenceCLS-HooshvareLab', endpoint='https://huggingface.co', repo_type='model', repo_id='MVesalA/peft-NER-existenceCLS-HooshvareLab'), pr_revision=None, pr_num=None)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "MVesalA/peft-NER-existenceCLS-HooshvareLab",  # "HooshvareLab/bert-fa-base-uncased"
    num_labels=2,  # Number of output labels (2 for binary sentiment classification)
    id2label=id2label,  # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id=label2id,  # {"NEGATIVE": 0, "POSITIVE": 1}
    quantization_config=bnb_config  # configuration for quantization
)

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:231: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/199M [00:00<?, ?B/s]

In [ ]:
def predict(input_text):
    """
    Predicts the sentiment label for a given text input.

    Args:
        input_text (str): The text to predict the sentiment for.

    Returns:
        float: The predicted probability of the text being positive sentiment.
    """
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # Convert to PyTorch tensors and move to GPU (if available)
    with torch.no_grad():
        outputs = model(**inputs).logits  # Get the model's output logits
    y_prob = torch.sigmoid(outputs).tolist()[0]  # Apply sigmoid activation and convert to list
    return np.round(y_prob, 5)  # Round the predicted probability to 5 decimal places

In [ ]:
df_test = pd.DataFrame(dataset['validation'])

df_test['prediction'] = df_test['text'].map(predict)
df_test['y_pred'] = df_test['prediction'].apply(lambda x: np.argmax(x, axis=0))
accuracy = (df_test['y_pred'] == df_test['existence']).mean()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
df_test.head()

Model Accuracy on Test Data: 0.7657


,text,existence,prediction,y_pred
0,نخستین مقاومت اسلامی لبنان در حوزه ادبیات کودک...,0,"[0.49414, 0.4707]",0
1,اما او نتوانست موافقت والدینش را برای خروج از ...,0,"[0.42505, 0.44165]",1
2,بر اساس دستور کار هفتگی مجلس ، قرار بود این اع...,0,"[0.52979, 0.4104]",0
3,در ادامه همین اقدامات مالی مشکوک ، مه آفرید ام...,1,"[0.37793, 0.55713]",1
4,چارلتون حریف اورتون نشد .,0,"[0.46997, 0.50146]",1
